### prereqs

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd /content/gdrive/MyDrive/
%rm -rf hosa-voice
! git clone https://github.com/marios1861/hosa-voice.git
%cd hosa-voice
! pip install poetry
! poetry config virtualenvs.in-project true
! poetry install --no-ansi

In [ ]:
import os, sys

VENV_PATH = "/content/gdrive/MyDrive/hosa-voice/.venv/lib/python3.10/site-packages"
LOCAL_VENV_PATH = '/content/venv' # local notebook
os.symlink(VENV_PATH, LOCAL_VENV_PATH) # connect to directory in drive
sys.path.insert(0, LOCAL_VENV_PATH)

### Lets first import and change the torchvision models' to output 4 classes instead of 1000

In [1]:
from torchvision.models import vit_b_16, ViT_B_16_Weights, mobilenet_v3_large, MobileNet_V3_Large_Weights

### Choose which of the two models to use

In [ ]:
model = vit_b_16(weights=ViT_B_16_Weights.DEFAULT)
# model = mobilenet_v3_large(weights=MobileNet_V3_Large_Weights.DEFAULT)

### Choose training lr and batch size

In [ ]:
lr = 0.001
batch_size = 32

### Initialize lightning module and data module

In [7]:
import os
from hosa_voice.voice_classifier import VoiceClassifier
from hosa_voice.voice_datasets import DatasetModule

pl_module = VoiceClassifier(model, lr)
os.chdir('..') # change to root directory
pl_data = DatasetModule(batch_size, data_dir='datasets/bicepstrum_image/bicepstrum_ml_normalized_imagesc_100_100')



/home/shinydemon/.cache/pypoetry/virtualenvs/hosa-voice-_kgzsXNz-py3.10/lib/python3.10/site-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
/home/shinydemon/.cache/pypoetry/virtualenvs/hosa-voice-_kgzsXNz-py3.10/lib/python3.10/site-packages/torchvision/transforms/v2/__init__.py:54: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking chan

NameError: name 'model' is not defined

### Initialize the trainer

In [ ]:
import torch
from lightning.pytorch import Trainer
from lightning.pytorch.loggers import TensorBoardLogger, CSVLogger
torch.set_float32_matmul_precision("medium")

epochs = 8
log_name = "transformer"
# log_name = "mobilenet"


# torch._dynamo.config.verbose=True
trainer = Trainer(
    # precision="16-mixed",
    # gradient_clip_algorithm="norm",
    max_epochs=epochs,
)

In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard
%tensorboard --logdir logs/tb_logs

In [ ]:
trainer.logger = TensorBoardLogger("logs", name=log_name, sub_dir="tb_logs")
trainer.fit(pl_module, pl_data)

In [ ]:
trainer.logger = CSVLogger("logs", name=log_name)
trainer.test(pl_module, pl_data)